In [1]:
import pandas as pd
import numpy as np

In [2]:
coAllData = pd.DataFrame()

In [3]:
def dailySummaryof21years(filename):
  df2007 = pd.read_csv(filename, encoding="utf8")
  df2007['month'] = pd.DatetimeIndex(df2007['Date Local']).month
  df2007['day'] = pd.DatetimeIndex(df2007['Date Local']).day
  df2007['year'] = pd.DatetimeIndex(df2007['Date Local']).year
  df2007 = df2007.drop(columns='Method Code')
  df2007 = df2007.dropna()
  df2007 = df2007.drop_duplicates()
  return df2007

In [17]:
coAllData = coAllData.append(dailySummaryof21years("/content/drive/MyDrive/FA21 CMPE 255 Term Project/CO/daily_42101_2018.csv"))

In [35]:
coAllData.to_csv('CO_12yearDailySummary.csv')

In [19]:
from sklearn import tree, preprocessing
le = preprocessing.LabelEncoder()
coAllData['Event Type'] = le.fit_transform(coAllData['Event Type'])

In [21]:
coAllData = coAllData.drop(columns=['Observation Percent'])
coAllData = coAllData.select_dtypes(exclude=['object'])                    

In [22]:
X_train = coAllData.drop(['AQI'], axis=1).values
Y_train = coAllData['AQI'].values

In [23]:
from sklearn.ensemble import GradientBoostingRegressor
grb = GradientBoostingRegressor(n_estimators=20)
grb.fit(X_train, Y_train)
print(grb.score(X_train, Y_train))

0.978524088394148


In [24]:
testdata = pd.DataFrame()

In [26]:
testdata = testdata.append(dailySummaryof21years("/content/drive/MyDrive/FA21 CMPE 255 Term Project/CO/daily_42101_2020.csv"))

In [27]:
testdata['Event Type'] = le.fit_transform(testdata['Event Type'])
testdata = testdata.drop(columns=['Observation Percent'])
testdata = testdata.select_dtypes(exclude=['object'])

In [28]:
X_test = testdata.drop(['AQI'], axis=1).values
Y_test = testdata['AQI'].values

In [29]:
accuracy = grb.score(X_test, Y_test)
print("Accuracy on test data: %.6f" % accuracy)

Accuracy on test data: 0.948245


In [30]:
predicted_AQI = grb.predict(X_test)

In [34]:
pd.DataFrame(predicted_AQI).value_counts()

2.411260     42005
3.189726     35258
5.067057     25539
5.911729     17127
1.595691     15118
6.818349     11748
7.630514      7607
8.555439      4980
9.381076      3301
1.015307      3216
10.242508     2198
12.047488     1499
12.941967     1033
13.768591      764
14.738028      524
15.546010      378
16.423504      220
17.190850      197
18.066320      108
19.970862       84
20.733779       70
21.459585       58
24.352988       47
22.838096       31
29.137729       28
23.156077       27
26.466800       16
27.174696       16
30.919807        9
59.501181        8
33.668055        4
32.147751        3
44.067704        3
48.375964        3
36.092426        2
39.753590        1
56.742382        1
dtype: int64

In [31]:
predictedResult = testdata.drop(['AQI'], axis=1)
predictedResult = predictedResult.reset_index()

In [32]:
predictedResult = predictedResult.join(pd.DataFrame(predicted_AQI))

In [33]:
countyResult = predictedResult.groupby(['County Code','year','month']).agg({0:['mean']})
countyResult.columns = ['AQI']
countyResult.reset_index()

,County Code,year,month,AQI
0,1,2019,1,5.955464
1,1,2019,2,5.191569
2,1,2019,3,4.948993
3,1,2019,4,4.367832
4,1,2019,5,4.028365
...,...,...,...,...
1780,760,2020,8,3.324680
1781,760,2020,9,3.490963
1782,760,2020,10,4.365803
1783,760,2020,11,6.644720
